In [1]:
import numpy as np
from numba import cuda
import time
import matplotlib.pyplot as plt

In [2]:
p = np.ones([2,2]) * 2 

In [3]:
stream = cuda.stream()

In [4]:
A0 = cuda.to_device(t, stream)

NameError: name 't' is not defined

In [5]:
A1 = cuda.to_device(p, stream)

In [ ]:
dt.device_ctypes_pointer

In [ ]:
dp.device_ctypes_pointer

In [ ]:
tmp1 = A0
tmp2 = A1

In [ ]:
A0 = tmp2
A1 = tmp1

Måste göra en separat arrat för ZZZZZ :@

In [ ]:
dt = dp

In [ ]:
dt.get_ipc_handle()

In [ ]:
dt.device_ctypes_pointer

In [ ]:
dt.setitem([0,0],2,stream)

In [ ]:
dt

In [ ]:
dt.copy_to_device(dp)

In [ ]:
stream.synchronize()

In [ ]:
dt.to_host(stream)

In [ ]:
t

In [ ]:
T.shape

In [ ]:
p

In [ ]:
from utils import significance_of_mean

### Synthetic data

In [ ]:
listsizes = [20,60,120,160,200]
args_list = list()
for ls in listsizes:
    x = list(np.random.randint(0,400,ls))
    y = list(np.random.randint(0,200,ls))
    m = len(x)
    n = len(y)
    z = x + y;z.sort()
    S = sum(z[m:])
    dtype = np.uint16
    args_list.append([m,n,S,z,dtype])


In [ ]:
np.random.seed(1)
#A = np.asarray([np.random.beta(2.0,5.0,7) for _ in range(1)])
#B = np.asarray([np.random.beta(2.0,5.0,20) for _ in range(1)])
A = np.asarray([np.random.randint(0,5,2) for _ in range(1)])
B = np.asarray([np.random.randint(0,5,2) for _ in range(1)])


In [ ]:
args_list = list()
x = list(A[0])
y = list(B[0])
m = len(x)
n = len(y)
z = x + y;z.sort()
S = sum(z[m:])
dtype = np.uint16
args_list.append([m,n,S,z,dtype])

In [ ]:
num_bin = np.ceil(max(z)) - np.floor(min(z)) + 1

### Mimic R's "table(factor(x, y))" function

In [ ]:
def table(val, S):
    table = np.zeros(S + 1)
    for e in val:
        table[e] +=1
    return table

### Shift-function 

In [ ]:
def shift(v, i): return np.pad(v[0:len(v) - i], (i,0), 'constant')

# Exact permutation test with the shift-method

In [ ]:
def exact_perm_shift1(m, n, S, z, dtype):
    N = np.zeros([S + 1, m + n, m], dtype)
    for k in range(1,(m+n)+1):
        N[:,k -1,0] = table(z[0:k],S)
    
    for k in range(2,(m+n)+1):
        for j in range(2, m +1):
            N[:,k-1,j-1] = shift(N[:,k-2,j-2], z[k -1]) + N[:,k-2,j-1]
    return N[:,-1,-1]

In [ ]:
def exact_perm_shift2(m, n, S, z, dtype):
    init = np.zeros([S + 1, m + n], dtype)
    for k in range(1,(m+n)+1):
        init[:,k -1] = table(z[0:k],S)

    
    N = np.zeros([S + 1, m], dtype)
    N_old = N.copy()
    for k in range(1,(m+n)+1):
        if k==1:
            N[:,0] = init[:,0]
        else:
            for j in range(1, m +1):
                if j == 1:
                    N[:,j-1] = init[:,k-1]
                else:
                    #N[:,k-1] = shift(N_old[:,k-2], z[k -1]) + N[:,k-2]
                    N[:,j-1] = shift(N_old[:,j-2], z[k -1]) + N_old[:,j-1]
        #print("-------",k,"--------------")
        #print(N)
        N_old = N.copy()
        #N = np.zeros([S + 1, m], dtype)

    return N_old[:,-1]

In [ ]:
def exact_perm_shift2(m, n, S, z, dtype):
    N = np.zeros([S + 1, m], dtype)
    N_old = N.copy()
    
    for k in range(1,(m+n)+1):
        if k==1:
            N[z[k-1],0] += 1

        else:
            for j in range(1, m +1):
                if j == 1:
                    N[z[k-1],0] += 1
                else:
                    for i in range(S+1):        
                        if i >= z[k-1]:
                            #print(i,i - z[k -1], j-1, N_old[i - z[k -1], j-2])
                            N[i,j-1] = N_old[i - z[k -1], j-2] + N_old[i,j-1]
                        else:
                            #print(i,j-1, N_old[i,j-1])
                            N[i,j-1] = N_old[i,j-1]
                            #N[i,j-1] =0
        #print(N)
        N_old = N.copy()
        print(N)
        #N = np.zeros([S + 1, m], dtype)

    return N_old[:,-1]

USE THIS! But have to explain why the non-zero verison works though!!!!!

In [ ]:
def exact_perm_shift3(m, n, S, z, dtype):
    N = np.zeros([S + 1, m], dtype)
    N_old = N.copy()
    
    for k in range(1,(m+n)+1):
        if k==1:
            N[z[k-1],0] += 1

        else:
            for j in range(1, m +1):
                for i in range(S+1):   
                    if j == 1:
                        if i == z[k-1]:
                            N[z[k-1],0] =  N_old[z[k-1],0] + 1
                        else:
                            N[i,0] = N_old[i,0]
                            
                    else:
                        if i >= z[k-1]:
                            #print(i,i - z[k -1], j-1, N_old[i - z[k -1], j-2])
                            N[i,j-1] = N_old[i - z[k -1], j-2] + N_old[i,j-1]
                        else:
                            #print(i,j-1, N_old[i,j-1])
                            N[i,j-1] = N_old[i,j-1]
                            #N[i,j-1] =0
        
        print(N)
        tmp = N.copy()
        N = N_old
        N_old = tmp
        #print(N)
        #N = np.zeros([S + 1, m], dtype)

    return N_old[:,-1]

In [ ]:
exact_perm_shift2(m, n, S, z, dtype)

In [ ]:
exact_perm_shift3(m, n, S, z, dtype)

In [ ]:
exact_perm_shift2(m, n, S, z, dtype) == exact_perm_shift3(m, n, S, z, dtype)

In [ ]:
def p_value_calc(args):
    a,b = args
    p=significance_of_mean(a,b, bins)[0]
    return p

def calibration_series_generator(A,B):
    num_tests = A.shape[0]
    for i in range(num_tests):
        a_sample = A[i].tolist()
        b_sample = B[i].tolist()
        yield ([a_sample,b_sample])

def calibration_test(A,B):
    with cf.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()-1) as pool:
        p_list = list(pool.map(p_value_calc, calibration_series_generator(A,B)))
    return p_list

In [ ]:
np.random.randint(0,20,20)

In [ ]:
np.random.beta(2.0, 5.0, 20)

### A function that calculates the permutations of the sub-array on the GPU

In [ ]:
@cuda.jit("(u2[:,:,:], u2, u2)")
def get_perumations(X, k_, z_):
    n = X.shape[0]
    m = X.shape[2]
    
    i = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    j = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y
    if j >= 2 and j < m + 1 and  i < n :
        if i >= int(z_):
            X[i, k_- 1, j-1] = X[i - int(z_), k_-2 ,j-2] +  X[i,k_-2,j-1]
        else:
            X[i, k_-1, j-1] = X[i,k_-2, j-1]

# Exact permutation test with a parallelized(on the GPU) version of the shift-method

In [ ]:
def exact_perm_numba_shift(m, n, S, z, dtype):
    N_cuda = np.zeros([S + 1, m + n, m], dtype)
    NN, NM = N_cuda[:,0,:].shape

    for k in range(1,(m+n)+1):
        N_cuda[:,k -1,0] = table(z[0:k],S )
    
    A = N_cuda.copy()
    blockdim = (256, 3)
    griddim = (int(np.ceil((NN )/ blockdim[0])), int(np.ceil(NM/blockdim[1] + 1)))
    A = np.ascontiguousarray(A) 
    stream = cuda.stream()
    dA = cuda.to_device(A, stream)
    
    for k in range(2, (m+n)+1):
        dk = dtype(k) 
        dz = dtype(z[k - 1])
        get_perumations[griddim,blockdim, stream](dA, dk, dz)
    dA.to_host(stream)
    stream.synchronize()
    return A


In [ ]:
plain_shift = list()
gpu_shift = list()
for args in args_list:
    start = time.time()
    N1 = exact_perm_shift1(*args)
    end = time.time()
    plain_shift.append(round(end - start,3))
    print("Plain")
    print(round(end - start,3))
    
    start = time.time()
    N2 = exact_perm_shift2(*args)
    end = time.time()
    plain_shift.append(round(end - start,3))
    print("Plain")
    print(round(end - start,3))
    
    #start = time.time()
    #A = exact_perm_numba_shift(*args)
    #end = time.time()
    #gpu_shift.append(round(end - start,3))
    #print("GPU")
    #print(round(end - start,3))
    
    #print(np.allclose(A[:,:,-1], N[:,:,-1]))
    print(np.allclose(N2, N1))
    
    

In [ ]:
plt.plot(listsizes, plain_shift, 'r-', label='Plain shift-method')
plt.plot(listsizes, gpu_shift, 'g-', label='GPU shift-method')
plt.legend(loc='upper left')
plt.xlabel("Sample size")
plt.ylabel("Time(s)")
plt.savefig("figures/comparison")
plt.show()